In [1]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 平開法

* "Napier's bone"
  * 位取り記法に依存
  * 条件分岐によるループ終了判定
  * 収束が遅い

## Newton法による平方根

In [2]:
from fractions import Fraction

In [3]:
def f(x):
    return (x + 2/x) / 2
    #return Fraction((x + Fraction(2,x)), 2)

x, c = 2, 4
for i in range(c):
    x = f(x)
    print(x, x**2)

1.5 2.25
1.4166666666666665 2.006944444444444
1.4142156862745097 2.0000060073048824
1.4142135623746899 2.0000000000045106


In [4]:
def f(x):
    #return (x + 2/x) / 2
    return Fraction((x + Fraction(2,x)), 2)

x, c = 2, 4
for i in range(c):
    x = f(x)
    print(x, x**2)

3/2 9/4
17/12 289/144
577/408 332929/166464
665857/470832 443365544449/221682772224


In [5]:
float(577/408)

1.4142156862745099

In [6]:
from fractions import Fraction

def f(x):
    #return (x + 46785399/x) / 2
    return Fraction((x + Fraction(46785399,x)), 2)

x, c = 7000, 3
for i in range(c):
    x = f(x)
    print(x, x**2)

95785399/14000 9174842661589201/196000000
18344780865589201/2681991172000 336530985006487674686224887818401/7193076646685933584000000
673061945959271105101164903818401/98401080667569511321067144000 453012383098480777483579996941511970856316830970796325229986196801/9682772676545522223779519889679433739577278956316736000000


In [7]:
from fractions import Fraction

def f(x):
    return (x + 46785399/x) / 2
    #return Fraction((x + Fraction(46785399,x)), 2)

x, c = 7000, 4
for i in range(c):
    x = f(x)
    print(x, x**2)

6841.814214285714 46810421.742802046
6839.9855514472965 46785402.344007775
6839.985307001767 46785399.00000006
6839.985307001763 46785399.00000001


## ニュートン法の改善

* [Methods of computing square roots - Wikipedia](https://en.wikipedia.org/wiki/Methods_of_computing_square_roots#A_two-variable_iterative_method)
* [High order algorithms to find square roots and inverses](http://numbers.computation.free.fr/Constants/Algorithms/inverse.html)

$f(x) = \frac{1}{x^2} - A$ とする. ニュートン法により

$$
\begin{eqnarray}
  x_{n+1} &=& x_{n} - \frac{{x_{n}}^{-2}-A}{-2{x_{n}}^{-3}} \\
          &=& x_{n} + \frac{1}{2} x_{n} \left( 1 - A {x_{n}}^2 \right) \\
          &=& x_{n} - \frac{1}{2} x_{n} c_{n} \\
          &=& x_{n} \left( 1-\frac{c_n}{2} \right)
\end{eqnarray}
$$

右辺を変形中に $c_{n} = A{x_{n}}^2-1$ とおいて二変数漸化式とした。 
これを $\frac{1}{A} (1+c_{n}) = {x_{n}}^2$ と変形し、上式の両辺を自乗して代入すると

$$(1+c_{n+1}) = (1+c_{n}) \left( 1-\frac{c_{n}}{2}\right)^{2}$$

$$
\begin{eqnarray}
c_{n+1} &=& (1+c_{n})(1-c_{n}+\frac{{c_{n}}^2}{4}) - 1 \\
        &=& \frac{1}{4}{c_{n}}^2 \left( c_{n} - 3 \right)
\end{eqnarray}
$$

あらためて、$S=\frac{1}{A}$ とおくと、$x_{n}$ は、初項を $0 < S < 3$ としたとき $\sqrt{S}$ に収束し、同時に $c_{n}$ は、初項を $S-1$ としたとき、$0$ に収束する。

$$
  S (1+c_{n}) = {x_{n}}^2
$$

なので、$c_{n} \rightarrow 0$ ならば ${x_{n}}^2 \rightarrow S$

In [156]:
def sqrt_edsac(x, c):
    return (x - x*c/2), (c**2 * (c-3))/4

In [157]:
x = 2
c = x - 1
for _ in range(10):
    x, c = sqrt_edsac(x, c)
    print(x, c)

1.0 -0.5
1.25 -0.21875
1.38671875 -0.03850555419921875
1.413416936993599 -0.001126281109816385
1.4142128893918142 -9.517390282169533e-07
1.4142135623726146 -6.793555988965238e-13
1.414213562373095 -3.461430223141193e-25
1.414213562373095 -8.986124392256467e-50
1.414213562373095 -6.056282369482999e-99
1.414213562373095 -2.7508917104182954e-197


### EDSAC法により $46785399$ の平方根を求める

In [8]:
import math
math.sqrt(46785399)

6839.9853070017625

1) 初項 $s$ を $0<s<3$ に納める

In [142]:
s = 46785399
s>>24, 2**24, s/2**24

(2, 16777216, 2.7886270880699158)

In [158]:
x = 46785399 / 2**24
c = x - 1
for _ in range(10):
    x, c = sqrt_edsac(x, c)
    print(x, c)

0.29472011394622477 -0.9688520756554101
0.4374902110133242 -0.9313649051351091
0.6412217254523059 -0.8525563697810122
0.9145605586904734 -0.7000599258715721
1.2346841571514315 -0.4533349495016725
1.5145468971678113 -0.1774259406953237
1.6489068511464027 -0.025006313899299287
1.6695233923016388 -0.00047289601151055086
1.6699181477783103 -1.6774941678609557e-07
1.6699182878422079 -2.1104901304167963e-14


In [159]:
x, x**2, x * 2**12

(1.6699182878422079, 2.788627088069851, 6839.985307001683)

2) 初項 $s$ を $0<s<3$ に納める

In [160]:
s = 46785399
s>>26, 2**26, s/2**26

(0, 67108864, 0.6971567720174789)

In [161]:
x = 46785399 / 2**26
c = x - 1
for _ in range(10):
    x, c = sqrt_edsac(x, c)
    print(x, c)

0.8027213756413029 -0.07572925807380614
0.8331161227499533 -0.004409765715489015
0.8349530462075152 -1.4605963362266851e-05
0.8349591438543162 -1.6000140329266338e-10
0.8349591439211136 -1.920033679274016e-20
0.8349591439211136 -2.7648969971598867e-40
0.8349591439211136 -5.733491553677818e-80
0.8349591439211136 -2.465469404707116e-159
0.8349591439211136 -4.5589e-318
0.8349591439211136 -0.0


In [162]:
x, x**2, x * 2**13

(0.8349591439211136, 0.6971567720174788, 6839.9853070017625)

$2$ や $4$ (や$2$の冪数) で割るのはコンピュータにとって最小コスト

In [74]:
x=123456789
x/2, x>>1, bin(x), bin(x>>1)

(61728394.5,
 61728394,
 '0b111010110111100110100010101',
 '0b11101011011110011010001010')

In [75]:
x=123456789
x/4, x>>2, bin(x), bin(x>>2)

(30864197.25,
 30864197,
 '0b111010110111100110100010101',
 '0b1110101101111001101000101')